# Práctica 2 - Visión por Computador

## Import all libraries

### Importar Genérico

In [0]:
# Import NumPy
import  numpy  as np
# Import Plot
import  matplotlib.pyplot as plt

# Import Enumerators
import enum

# Keras Imports
import keras
import keras.utils as  np_utils

# Import Image Processing Keras
from keras.preprocessing.image import load_img, img_to_array

# Import Keras Sequential Model
from keras.models import Sequential

# Import Keras Convolutional Layers
from keras.layers.convolutional import Conv2D, MaxPooling2D

# Import Keras Common Layers
from keras.layers import Activation, Dense, Dropout, Flatten, BatchNormalization

# Import Keras Image Generators (IGenerator)
from keras.preprocessing.image import ImageDataGenerator  

# Import Keras Optimizers
from keras.optimizers  import  SGD

# Import Keras Callbacks
from keras.callbacks import EarlyStopping

# Import Cifar Dataset
from keras.datasets  import  cifar100

# Import ResNet50 
from keras.applications.resnet50 import ResNet50

# Import InceptionResNetV2
from keras.applications.inception_resnet_v2 import InceptionResNetV2

DEBUG = False

### Importar Drive

In [2]:
# Import Google Drive Library
from google.colab import drive
# Mount in the colab fs our google drive
drive.mount("/content/drive", force_remount = True)

Mounted at /content/drive


In [0]:
# Zip Path
file_name = "CUB-200"
zip_path = "/content/drive/My Drive/VC/"+ file_name +".zip"
# Mount Path (best way temp folder so we dont loose drive's space)
to_path="/tmp"
# Remove if we want to rebuild
!rm -rf "$to_path"
# Unzip into the "mount path"
!unzip -q "$zip_path" -d "$to_path"
# Rename with the created folder
to_path = to_path + "/" + file_name

## Cargar Imágenes

In [0]:
class ILoader(enum.Enum):
   CIFAR  = 0
   CALTECH = 1

def load_dataset(iload_type = ILoader.CIFAR):

  print("--- Loading Dataset ---")

  data_set = ""

  value = int(iload_type.value)

  if   value == int(ILoader.CIFAR.value)   : data_set = load_cifar_dataset()
  elif value == int(ILoader.CALTECH.value) : data_set = load_caltech_dataset(to_path)
  
  return data_set


### Cargar CIFAR

In [0]:
def load_cifar_dataset():

  (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  
  x_train /= 255
  x_test  /= 255
  
  train_idx = np.isin(y_train, np.arange(25))
  train_idx = np.reshape(train_idx, -1)
  x_train = x_train[train_idx]
  y_train = y_train[train_idx]
  
  test_idx = np.isin(y_test , np.arange(25))
  test_idx = np.reshape(test_idx, -1)
  x_test = x_test[test_idx]
  y_test = y_test[test_idx]

  y_train = np_utils.to_categorical(y_train, 25)
  y_test = np_utils.to_categorical(y_test, 25)
  
  return  x_train, y_train, x_test, y_test

### Cargar Caltech-UCSD

In [0]:

def read_images(vec_imagenes, path):
  clases = np.array([img.split('/')[0] for img in vec_imagenes])
  imagenes = np.array([
    img_to_array(
      load_img(
        path + "/" + img, 
        target_size = (224, 224),
        interpolation = 'nearest'
      )
    ) for img in vec_imagenes
  ])
  return imagenes, clases

def load_caltech_dataset(path):
  train_images = np.loadtxt(path + "/train.txt", dtype = str)
  test_images = np.loadtxt(path + "/test.txt", dtype = str)

  x_train, y_train = read_images(train_images, path + "/images")
  x_test, y_test = read_images(test_images, path + "/images")
  
  clases_posibles = np.unique(np.copy(y_train))
  for i in range(len(clases_posibles)):
    y_train[y_train == clases_posibles[i]] = i
    y_test[y_test == clases_posibles[i]] = i

  y_train = np_utils.to_categorical(y_train, 200)
  y_test = np_utils.to_categorical(y_test, 200)
  
  train_perm = np.random.permutation(len(x_train))
  x_train = x_train[train_perm]
  y_train = y_train[train_perm]

  test_perm = np.random.permutation(len(x_test))
  x_test = x_test[test_perm]
  y_test = y_test[test_perm]
  
  return x_train, y_train, x_test, y_test

## Evolución

In [0]:
def  show_evolution(hist):
  loss = hist.history['loss']
  val_loss = hist.history['val_loss']
  plt.plot(loss)
  plt.plot(val_loss)
  plt.legend(['Training  loss', 'Validation  loss'])
  plt.show()

  acc = hist.history['acc']
  val_acc = hist.history['val_acc']
  plt.plot(acc)
  plt.plot(val_acc)
  plt.legend(['Training  accuracy','Validation  accuracy'])
  plt.show()

## Modelos

Hablar sobre las escalas (características)

### Modelo Práctica 1

In [0]:
def BaseNet():
  return Sequential([
    # 5 Kernel = 4 + 1 ( central ) = 2 (left/right, top/bottom)
    # 32 - 2 (left/bottom) - 2 (right/top) = 28
    # Input Image 32 x 32 x 3
    Conv2D(6, kernel_size = (5, 5), input_shape = (32, 32, 3)),
    
    Activation("relu"),
    
    # 28 / 2 = 14
    MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    
    # 14 - 2 (left/bottom) - 2 (right/top) = 10
    Conv2D(16, kernel_size = (5, 5)),
    
    Activation("relu"),
    
    # 10 / 2 = 5
    MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    
    # Aplanar antes de pasarlo por las capas dense
    Flatten(),

    # Input 400, output 50
    Dense(units = 50, input_shape = (400,)),
    
    Activation("relu"),
    
    Dense(units = 25, input_shape = (50,)),

    Activation("softmax")
  ])

### Modelo Práctica 2

#### Modelo BatchNormalization Antes

In [0]:
def BaseNetBatchBefore():
  return Sequential([
    # 5 Kernel = 4 + 1 ( central ) = 2 (left/right, top/bottom)
    # 32 - 2 (left/bottom) - 2 (right/top) = 28
    # Input Image 32 x 32 x 3
    Conv2D(6, kernel_size = (5, 5), input_shape = (32, 32, 3)),
    BatchNormalization(),
    Activation("relu"),
    
    # 28 / 2 = 14
    MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    
    # 14 - 2 (left/bottom) - 2 (right/top) = 10
    Conv2D(16, kernel_size = (5, 5)),
    BatchNormalization(),
    Activation("relu"),
    
    # 10 / 2 = 5
    MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    
    # Aplanar antes de pasarlo por las capas dense
    Flatten(),

    # Input 400, output 50
    Dense(units = 50, input_shape = (400,)),
    
    Activation("relu"),
    
    Dense(units = 25, input_shape = (50,)),

    Activation("softmax")
  ])

#### Modelo BatchNormalization Después

In [0]:
def BaseNetBatchAfter():
  return Sequential([
    # 5 Kernel = 4 + 1 ( central ) = 2 (left/right, top/bottom)
    # 32 - 2 (left/bottom) - 2 (right/top) = 28
    # Input Image 32 x 32 x 3
    Conv2D(6, kernel_size = (5, 5), input_shape = (32, 32, 3)),
    Activation("relu"),
    BatchNormalization(),

    # 28 / 2 = 14
    MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    
    # 14 - 2 (left/bottom) - 2 (right/top) = 10
    Conv2D(16, kernel_size = (5, 5)),
    Activation("relu"),
    BatchNormalization(),
    
    # 10 / 2 = 5
    MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    
    # Aplanar antes de pasarlo por las capas dense
    Flatten(),

    # Input 400, output 50
    Dense(units = 50, input_shape = (400,)),
    Activation("relu"),

    Dense(units = 25, input_shape = (50,)),
    Activation("softmax")
  ])

#### Modelo Optimizado

In [0]:
def BaseNetBatchOptimized():
  return Sequential([
    # 5 Kernel = 4 + 1 ( central ) = 2 (left/right, top/bottom)
    # Input Image 32 x 32 x 3
    # 32 - 3 (left/bottom) - 3 (right/top) = 26
    Conv2D(16, kernel_size = (3, 3), input_shape = (32, 32, 3)),
    Activation("relu"),
    BatchNormalization(),

    # 26 - 2 (left/bottom) - 2 (right/top) = 22
    Conv2D(32, kernel_size = (3, 3)),
    Activation("relu"),
    BatchNormalization(),

    # 26 / 2 = 11
    MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    
    # 11 - 2 (left/bottom) - 1 (right/top) = 9
    Conv2D(64, kernel_size = (3, 3)),
    Activation("relu"),
    BatchNormalization(),

    # 13 - 1 (left/bottom) - 1 (right/top) = 11
    Conv2D(128, kernel_size = (3, 3)),
    Activation("relu"),
    BatchNormalization(),

    # 11 - 1 (left/bottom) - 1 (right/top) = 9
    Conv2D(256, kernel_size = (3, 3)),
    Activation("relu"),
    BatchNormalization(),

    # 11 / 2 = 5.5
    MaxPooling2D(pool_size = (2, 2), strides = (2, 2)),
    
    # Aplanar antes de pasarlo por las capas dense
    Flatten(),

    # Input 400, output 50
    Dense(units = 50, input_shape = (400,)),
    Activation("relu"),

    Dense(units = 25, input_shape = (50,)),
    Activation("softmax")
  ])

### ResNet50 Estractor de Características

In [0]:
def ResNetExtractor():
  # Create ResNet 
  Res50 = ResNet50(
    weights = "imagenet", # Set Weights of imagenet
    include_top = False, # Remove Last Layer
    pooling = "avg", # Add Average Polling last layer
    input_shape = (224, 224, 3) # Input shape and channels
  )

  # Set all layers to no trainable because we only want the characteristics
  for layer in Res50.layers:
    layer.trainable = False

  # Create Net Characteristics Extractor
  return Sequential([
    Res50,

    # Dense
    Dense(500, input_shape = (2048,)),
    Activation("relu"),

    # Dense of 200 characteristics
    Dense(200, input_shape = (500,)),
    Activation("softmax")
  ])

### ResNet50 Fine Tuning

In [0]:
# Create Net Fine Tuning ( Difference: All Layers are trainable)
def ResNetFineTuning():
  return Sequential([
    ResNet50(
      weights = "imagenet", # Set Weights of imagenet
      include_top = False, # Remove Last Layer
      pooling = "avg", # Add Average Polling last layer
      input_shape = (224, 224, 3) # Input shape and channels
    ),

    # Dense
    Dense(500, input_shape = (2048,)),
    Activation("relu"),

    # Dense of 200 characteristics
    Dense(200, input_shape = (500,)),
    Activation("softmax")
  ])

### InceptionResNetV2 (Bonus)

In [0]:
# Create 
def Inception():
  return Sequential([
    InceptionResNetV2(
      weights = "imagenet", # Set Weights of imagenet
      include_top = False, # Remove Last Layer
      pooling = "avg", # Add Average Polling last layer
      input_shape = (224, 224, 3) # Input shape and channels
    ),
    # Dense
    Dense(1024, input_shape = (2048,)),
    Activation("relu"),
    # Dense of 200 characteristics
    Dense(200, input_shape = (1024,)),
    Activation("softmax")
  ])

## Generadores de Imágenes

In [0]:
class IGenerator(enum.Enum):
   SIMPLE    = 0
   COMPLEX   = 1
   WHITENING = 2
   RESNET    = 3
   INCEPTION = 4


def igenerator(igenerator_type = IGenerator.SIMPLE, split = 0.0):

  generator = ""

  value = int(igenerator_type.value)
  if   value == int(IGenerator.SIMPLE.value)    : generator = IGenerator_simple(split)
  elif value == int(IGenerator.COMPLEX.value)   : generator = IGenerator_complex(split)
  elif value == int(IGenerator.WHITENING.value) : generator = IGenerator_whitening(split)
  elif value == int(IGenerator.RESNET.value)    : generator = IGenerator_resnet(split)
  elif value == int(IGenerator.INCEPTION.value) : generator = IGenerator_inception(split)
  
  return generator

### Generador Simple

In [0]:
def IGenerator_simple(split = 0.0):
  return ImageDataGenerator(
    validation_split = split
  )

### Generador Mejorado

In [0]:
def IGenerator_complex(split = 0.0):
  return ImageDataGenerator(
    validation_split = split,
    featurewise_center = True,
    featurewise_std_normalization = True,
    horizontal_flip = True,
    vertical_flip = True,
    zoom_range = 0.2
  )

### Generador Whitening

In [0]:
def IGenerator_whitening(split = 0.0):
  return ImageDataGenerator(
    validation_split = split,
    featurewise_center = True,
    featurewise_std_normalization = True,
    zca_whitening = True
  )

#### Generador RESNET

In [0]:
def IGenerator_resnet(split = 0.0):
  return ImageDataGenerator(
    validation_split = split,
    preprocessing_function = keras.applications.resnet50.preprocess_input
  )

#### Generador RESNET-INCEPTION

In [0]:
def IGenerator_inception(split = 0.0):
  return ImageDataGenerator(
    validation_split = split,
    preprocessing_function = keras.applications.inception_resnet_v2.preprocess_input
  )

## Compiladores

In [0]:
class Compile(enum.Enum):
   SGD = 0
   ADAM = 1

def compiler(model, compile_type = Compile.SGD):

  optimizer = ""

  value = int(compile_type.value)
  if   value == int(Compile.SGD.value) : optimizer = sgd_optimizer() 
  elif value == int(Compile.ADAM.value): optimizer = adam_optimizer()

  model.compile(
    optimizer,
    loss = "categorical_crossentropy",
    metrics = ["accuracy"]
  )

  return model

### Optimizador SGD


In [0]:
def sgd_optimizer():
  return SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)

### <a href="https://arxiv.org/pdf/1712.07628.pdf">Optimizador Adam</a>

In [0]:
def adam_optimizer():
  return "adam";

## Entrenamiento

In [0]:
class TrainInfo(enum.Enum):
   NONE = 0
   FULL = 1
   HALF = 2

def train(model, batch_size = 32, epochs = 80, dataset = ILoader.CIFAR, info = TrainInfo.FULL, compile_type = Compile.SGD, igenerator_type = IGenerator.SIMPLE, early_stopping = False):
  # Detail of the Information
  verbose = int(info.value)

  # Callbacks
  callbacks = []  
  
  # Early Stopping Callback
  if early_stopping:
    callbacks.append(
      EarlyStopping(
        # Monitorize Val Loss
        monitor  = 'val_loss',
        # Patience (Minimum number of epochs at least trains)
        patience = 10,
        # Information Quantity
        verbose  = verbose,
      )
    )
  
  # Load Dataset
  x_train, y_train, x_test, y_test = load_dataset(dataset)

  # Debug
  if DEBUG: 
    debug(model, compile_type, igenerator_type, x_train, y_train)
  
  print("--- Training Model ---")
  # Compile Model
  compiler(model, compile_type)
  
  # Obtain Generator, pass images and standarize (data augmentation)
  generator_train = igenerator(
    igenerator_type,
    split = 0.1
  )

  # Asign to ram a size of x_train
  generator_train.fit(x_train)
  
  # Fits the model on batches
  history = model.fit_generator(
    # Training Data Batch
    generator = generator_train.flow(
      x_train,
      y_train,
      batch_size = batch_size,
      subset = "training"
    ),
    # Number of epochs (Times all batches will pass)
    epochs = epochs,
    # Number of batches for the training
    steps_per_epoch = len(x_train) * 0.9 / batch_size,

    # Validation Data Batch
    validation_data = generator_train.flow(
      x_train,
      y_train,
      batch_size = batch_size,
      subset = "validation"
    ),
    # Number of batches for the testing
    validation_steps = len(x_train) * 0.1 / batch_size,
    # Appearance the info will show per bach
    verbose = verbose,
    # Callbacks (Early Stopping, etc.)
    callbacks = callbacks
  )

  # Evaluate
  evaluate(model, x_test, y_test, info, igenerator_type)

  # Return Training History
  return history



## Evaluación

In [0]:
def evaluate(model, x_test, y_test, info = TrainInfo.FULL, igenerator_type = IGenerator.SIMPLE):
  # Detail of the Information
  verbose = int(info.value)

  print("--- Evaluate Results ---")

  # Test Generator
  generator_test = igenerator(
    igenerator_type
  )

  # Asign to ram  a size of x_test
  generator_test.fit(x_test)

  # Calc Accuracy
  scores = model.evaluate_generator(
    generator = generator_test.flow(
      x_test,
      y_test,
      batch_size = 1,
      shuffle = False
    ),
    # Appearance the info will show
    steps = len(x_test),
    verbose = verbose,
  )

  print("Accuracy: ", str(scores[1]))

## Debug

In [0]:
def debug(model, compile_type, igenerator_type, x_train, y_train):
  # Shows layer information of our model
  print("Compiler  Type {}".format(compile_type))
  print("Generator Type {}".format(igenerator_type))
  model.summary()
  
  # Used for show nth images from the classes
  rows = 5
  cols = 5
  for i in range(rows * cols):
    plt.subplot(rows, cols, i + 1)
    plt.imshow(x_train[i], cmap = 'gray', interpolation = 'none')
    plt.title("Class  {}".format(i))# y_train[i]

  plt.show()

## Tests

## Ejercicios

### Ejercicio 1 

Hay Overfitting (La gráfica de val_loss va creciendo)

In [0]:
# BaseNet
history = train(
  BaseNet(),
  info = TrainInfo.HALF,
  dataset = ILoader.CIFAR,
  epochs = 30,
  #early_stopping = True
)

show_evolution(history)

### Ejercicio 2

In [0]:
# BaseNet 
history = train(
  BaseNetBatchOptimized(),
  info = TrainInfo.FULL,
  compile_type = Compile.SGD,
  igenerator_type = IGenerator.COMPLEX,
  epochs = 30,
  #early_stopping = True
)

show_evolution(history)

In [0]:
# BaseNet Whitening
history = train(
  BaseNetBatchOptimized(),
  info = TrainInfo.FULL,
  compile_type = Compile.SGD,
  igenerator_type = IGenerator.SIMPLE,
  epochs = 30,
  #early_stopping = True
)

show_evolution(history)

In [0]:
# BaseNet Batch After
history = train(
  BaseNetBatchAfter(),
  info = TrainInfo.FULL,
  compile_type = Compile.SGD,
  igenerator_type = IGenerator.COMPLEX,
  epochs = 30,
  #early_stopping = True
)

show_evolution(history)

In [0]:
# BaseNet Batch Before
history = train(
  BaseNetBatchBefore(),
  info = TrainInfo.FULL,
  compile_type = Compile.SGD,
  igenerator_type = IGenerator.COMPLEX,
  epochs = 30,
  #early_stopping = True
)

show_evolution(history)

## Ejercicio 3

### Estractor de características

In [0]:
history = train(
  ResNetExtractor(),
  dataset = ILoader.CALTECH,
  igenerator_type = IGenerator.RESNET,
  info = TrainInfo.HALF,
  compile_type = Compile.SGD,
  epochs = 15,
  #early_stopping = True
)

show_evolution(history)

### Fine Tuning

In [0]:
history = train(
  ResNetFineTuning(),
  dataset = ILoader.CALTECH,
  igenerator_type = IGenerator.RESNET,
  info = TrainInfo.HALF,
  compile_type = Compile.SGD,
  epochs = 15,
  #early_stopping = True
)

show_evolution(history)

## Bonus
https://medium.com/nanonets/how-to-easily-build-a-dog-breed-image-classification-model-2fd214419cde

https://towardsdatascience.com/dog-breed-classification-hands-on-approach-b5e4f88c333e

https://ai.googleblog.com/2016/08/improving-inception-and-image.html

In [0]:
history = train(
  Inception(),
  dataset = ILoader.CALTECH,
  igenerator_type = IGenerator.INCEPTION,
  info = TrainInfo.HALF,
  compile_type = Compile.SGD,
  epochs = 10,
  #early_stopping = True
)

show_evolution(history)

## Resumen

| Model              | Compile | Generator | Epochs | Accuracy                 |
|--------------------|---------|-----------|--------|--------------------------|
| Normal             | SGD     | Simple    | 22     | t1: 0.4116               |
| Optimized          | SGD     | Simple    | 80     | 0.5092                   |
| Optimized          | ADAM    | Simple    | 80     | 0.498                    |
| Optimized          | SGD     | Complex   | 80     | t1: 0.6016, t2: 0.5876   |
| BaseNetBatchAfter  | SGD     | Simple    | 58     | t1: 0.488, t2:           |
| BaseNetBatchBefore | SGD     | Simple    | 80/38  | t1: 0.4996, t2: 0.4868   |